In [1]:
!pip install torch
!pip install torch_geometric

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Step 1: Define a simple graph
# We'll create a graph with 3 nodes (Node 0, Node 1, Node 2)
# and define their features and connections.

# Node features (e.g., age and income) for each of the 3 nodes
# Node 0 has features [1.0, 2.0]
# Node 1 has features [3.0, 4.0]
# Node 2 has features [5.0, 6.0]
x = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=torch.float)

# Edge index defines the connections (edges) in the graph.
# It's a list of [source_node, target_node] pairs.
# Connections are: 0->1, 1->0, 1->2, 2->1.
# This makes it a simple line graph: 0 -- 1 -- 2
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)

# Create the graph data object
data = Data(x=x, edge_index=edge_index)

print("--- Initial Graph Data ---")
print(f"Number of nodes: {data.num_nodes}")
print(f"Input node features:\n{data.x}\n")
print(f"Graph connections (edge index):\n{data.edge_index}\n")

# Step 2: Build a simple GNN model
# We'll use a single GCNConv layer (Graph Convolutional Network).
# This layer aggregates features from a node's immediate neighbors.
# The input features have a dimension of 2, and we'll transform them
# to an output dimension of 16.
in_channels = data.num_node_features
out_channels = 16
gnn_layer = GCNConv(in_channels, out_channels)

# Step 3: Pass the graph data through the GNN
# The forward pass updates the node features by incorporating information
# from their neighbors.
out = gnn_layer(data.x, data.edge_index)

print("--- After one GNN layer ---")
print(f"Shape of output features: {out.shape}")
print(f"Output node features:\n{out}\n")

# Step 4: Show the effect of the GNN layer
# Compare the output features of each node.
# For example, the output of Node 0 has been updated not just
# by its own features, but also by the features of its neighbor, Node 1.
# The same is true for Node 2, which gets information from Node 1.
# Node 1 gets information from both Node 0 and Node 2.
print("Output for Node 0:\n", out[0])
print("\nOutput for Node 1:\n", out[1])
print("\nOutput for Node 2:\n", out[2])

--- Initial Graph Data ---
Number of nodes: 3
Input node features:
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

Graph connections (edge index):
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

--- After one GNN layer ---
Shape of output features: torch.Size([3, 16])
Output node features:
tensor([[-1.7839,  0.7413, -0.2682, -0.5322, -1.8943, -1.1825,  0.2546,  0.5568,
         -0.7075, -0.8199,  0.4678,  0.9642, -0.6832, -0.6324,  0.0194,  0.2868],
        [-3.3629,  1.3403, -0.2902, -1.0587, -3.4157, -2.2101,  0.4532,  0.9949,
         -1.1880, -1.3268,  1.0559,  1.6987, -1.1883, -0.9917, -0.1857,  0.4802],
        [-3.5288,  1.3761, -0.1902, -1.1404, -3.5019, -2.3089,  0.4614,  1.0149,
         -1.1693, -1.2762,  1.2003,  1.7194, -1.1941, -0.9342, -0.3127,  0.4719]],
       grad_fn=<AddBackward0>)

Output for Node 0:
 tensor([-1.7839,  0.7413, -0.2682, -0.5322, -1.8943, -1.1825,  0.2546,  0.5568,
        -0.7075, -0.8199,  0.4678,  0.9642, -0.6832, -0.6324,  0.0194,  0.2868],


In [2]:
# Define the Graph Data

import torch
from torch_geometric.data import Data

# Define node features (e.g., each node has a feature vector of size 2)
# We have 4 nodes, so the shape is [4, 2]
x = torch.tensor([[2.0, 1.0], [3.0, 5.0], [5.0, 3.0], [6.0, 2.0]], dtype=torch.float)

# Define the graph's connectivity (edges)
# This is an undirected graph, so we define edges in both directions
# The list of tuples represents connections: (node_1, node_2)
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
                           [1, 0, 2, 1, 3, 2]], dtype=torch.long)

# Define the node labels (e.g., a node can be in one of 2 classes)
# The shape is [4,]
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)

# Create the PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, y=y)

print(data)
# Output: Data(x=[4, 2], edge_index=[2, 6], y=[4])

# Define the GNN Model

import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        # First GCN layer: takes in_channels and outputs hidden_channels
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Second GCN layer: takes hidden_channels and outputs out_channels (the number of classes)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply the first GCN layer with a ReLU activation function
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        # Apply the second GCN layer
        x = self.conv2(x, edge_index)
        
        # Return the final node embeddings (logits)
        return x

# Train the Model

# Create an instance of the model
# Our node features have a size of 2, we'll use a hidden layer of size 8, and there are 2 output classes
model = GNN(in_channels=2, hidden_channels=8, out_channels=2)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    # Set the model to training mode
    model.train()
    
    # Reset gradients
    optimizer.zero_grad()
    
    # Forward pass: get the node predictions
    out = model(data)
    
    # Calculate the loss using the predictions and true labels
    loss = criterion(out, data.y)
    
    # Backward pass: compute gradients
    loss.backward()
    
    # Update model parameters
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# Final predictions
model.eval()
with torch.no_grad():
    final_output = model(data)
    predicted_classes = final_output.argmax(dim=1)
    print("\nFinal Predictions:", predicted_classes)
    print("Actual Labels:", data.y)

Data(x=[4, 2], edge_index=[2, 6], y=[4])
Epoch: 000, Loss: 0.6933
Epoch: 010, Loss: 0.6640
Epoch: 020, Loss: 0.6521
Epoch: 030, Loss: 0.6479
Epoch: 040, Loss: 0.6455
Epoch: 050, Loss: 0.6448
Epoch: 060, Loss: 0.6432
Epoch: 070, Loss: 0.6423
Epoch: 080, Loss: 0.6406
Epoch: 090, Loss: 0.6386

Final Predictions: tensor([0, 0, 1, 1])
Actual Labels: tensor([0, 1, 0, 1])
